# MCP demo (VS Code) – Setup progetto

Questo notebook crea una demo **MCP client/server** *locale* e completamente **open/gratis**:

- **MCP server arXiv** (tool: `arxiv_search`)
- **MCP server OpenAlex** (tool: `openalex_search_works`)
- **Client di test** che chiama entrambi e mostra i risultati

> Nota: la modalità `stdio` richiede che i server girino come processi locali (perfetto per prototipare in VS Code).


## 0.1 – Crea cartelle progetto

In [74]:

from pathlib import Path

ROOT = Path(".").resolve()
SERVERS = ROOT / "servers"
CLI = ROOT / "cli"
VSCODE = ROOT / ".vscode"

for p in [SERVERS, CLI, VSCODE]:
    p.mkdir(parents=True, exist_ok=True)

ROOT


PosixPath('/Users/monica.costantini/Documents/workspace/mcp-demo')

## 0.2 – Scrivi i file dei due MCP server (arXiv + OpenAlex)

In [75]:
from pathlib import Path

arxiv_server = r'''from __future__ import annotations
import arxiv
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("arxiv")

@mcp.tool()
def arxiv_search(topic: str, max_results: int = 5):
    """Search arXiv by topic and return basic metadata."""
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance,
    )

    out = []
    for r in search.results():
        out.append({
            "title": r.title,
            "authors": [a.name for a in r.authors],
            "published": r.published.isoformat() if r.published else None,
            "pdf_url": r.pdf_url,
            "entry_id": r.entry_id,
            "summary": r.summary,
        })

    return out

if __name__ == "__main__":
    # Non usare print() su stdout in stdio: rompe JSON-RPC.
    mcp.run(transport="stdio")
'''

openalex_server = r'''from __future__ import annotations
import httpx
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("openalex")
BASE = "https://api.openalex.org"

@mcp.tool()
def openalex_search_works(query: str, per_page: int = 5, mode: str = "ai"):
    """
    mode:
      - "ai": filtra per risultati AI/RAG/LLM (più precisione)
      - "general": nessun filtro AI (più recall)
    """
    url = f"{BASE}/works"

    fetch_n = max(per_page * 10, 50)
    params = {"search": query, "per_page": fetch_n}

    r = httpx.get(url, params=params, timeout=30.0)
    r.raise_for_status()
    data = r.json()

    keywords_ai = (
        "retrieval",
        "generation",
        "rag",
        "language model",
        "llm",
        "agent",
        "benchmark",
        "transformer",
    )

    negative_rag_bio = (
        "mice", "mouse", "murine",
        "gene", "genes", "genetic", "immun", "chemotherapy",
        "hiv", "cancer", "tumor",
    )

    results = []
    for w in data.get("results", []):
        title = (w.get("title") or "")
        t = title.lower()

        # se la query contiene "rag", evitiamo il significato biologico (RAG gene)
        if "rag" in query.lower() and any(bad in t for bad in negative_rag_bio):
            continue

        if mode == "ai":
            if not any(k in t for k in keywords_ai):
                continue

        results.append({
            "id": w.get("id"),
            "doi": (w.get("doi") or "").replace("https://doi.org/", "") if w.get("doi") else None,
            "title": title,
            "publication_year": w.get("publication_year"),
            "cited_by_count": w.get("cited_by_count"),
            "primary_location": (
                (w.get("primary_location") or {})
                .get("source", {})
                .get("display_name")
            ),
            "openalex_url": w.get("id"),
        })

        if len(results) >= per_page:
            break

    return results


if __name__ == "__main__":
    mcp.run(transport="stdio")
'''

ROOT = Path(".").resolve()
(SERVERS := ROOT / "servers").mkdir(parents=True, exist_ok=True)

(SERVERS / "arxiv_server.py").write_text(arxiv_server, encoding="utf-8")
(SERVERS / "openalex_server.py").write_text(openalex_server, encoding="utf-8")

print("Creati:", SERVERS / "arxiv_server.py", "e", SERVERS / "openalex_server.py")


Creati: /Users/monica.costantini/Documents/workspace/mcp-demo/servers/arxiv_server.py e /Users/monica.costantini/Documents/workspace/mcp-demo/servers/openalex_server.py


## 0.3 – Scrivi un client CLI di test (chiama entrambi i server)

Questo client:
1) lista i tool disponibili sui due server
2) chiama `arxiv_search`
3) chiama `openalex_search_works`
4) stampa risultati separati + una mini-sintesi (senza LLM)


In [76]:
from pathlib import Path

client_cli = r'''import asyncio
import argparse
import json
import httpx
from fastmcp import Client
from fastmcp.client.transports import StdioTransport

OLLAMA_URL = "http://localhost:11434/api/generate"

async def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("question", help="Es: 'agentic RAG evaluation'")
    ap.add_argument("--k", type=int, default=3, help="Risultati per sorgente")
    ap.add_argument("--llm", action="store_true", help="Genera risposta finale con LLM locale (Ollama)")
    ap.add_argument("--model", type=str, default="llama3.2", help="Modello Ollama (es: llama3.2)")
    args = ap.parse_args()

    arxiv = Client(StdioTransport(command="python", args=["servers/arxiv_server.py"]))
    openalex = Client(StdioTransport(command="python", args=["servers/openalex_server.py"]))

    async with arxiv, openalex:
        tools_a = await arxiv.list_tools()
        tools_o = await openalex.list_tools()

        print("\n== Tools arXiv ==")
        for t in tools_a:
            print("-", t.name)

        print("\n== Tools OpenAlex ==")
        for t in tools_o:
            print("-", t.name)

        ax = await arxiv.call_tool("arxiv_search", {"topic": args.question, "max_results": args.k})
        ox = await openalex.call_tool("openalex_search_works", {"query": args.question, "per_page": args.k})

        def unpack(resp):
            content = None
            for key in ("data", "result", "content", "value"):
                if hasattr(resp, key):
                    v = getattr(resp, key)
                    if v is not None:
                        content = v
                        break
            if content is None:
                content = resp

            # lista di TextContent
            if isinstance(content, list) and content and hasattr(content[0], "text"):
                text = "\n".join(c.text for c in content if hasattr(c, "text"))
                try:
                    return json.loads(text)
                except Exception:
                    return text

            # singolo TextContent
            if hasattr(content, "text"):
                try:
                    return json.loads(content.text)
                except Exception:
                    return content.text

            if isinstance(content, (list, dict)):
                return content

            return content

        ax_items = unpack(ax)
        ox_items = unpack(ox)

        print("\n==============================")
        print("RISULTATI arXiv (preprint)")
        print("==============================")
        if isinstance(ax_items, str):
            print(ax_items)
        elif not ax_items:
            print("(nessun risultato)")
        else:
            for i, p in enumerate(ax_items, 1):
                print(f"{i}. {p.get('title')}")
                print(f"   PDF: {p.get('pdf_url')}\n")

        print("\n==============================")
        print("RISULTATI OpenAlex (panoramica + citazioni)")
        print("==============================")
        if isinstance(ox_items, str):
            print(ox_items)
        elif not ox_items:
            print("(nessun risultato)")
        else:
            for i, w in enumerate(ox_items, 1):
                print(f"{i}. {w.get('title')}")
                print(f"   Year: {w.get('publication_year')} | Cited by: {w.get('cited_by_count')}")
                print(f"   DOI: {w.get('doi')} | Source: {w.get('primary_location')}")
                print(f"   OpenAlex: {w.get('openalex_url')}\n")

        if not args.llm:
            print("\n=== Sintesi (senza LLM) ===")
            print("- arXiv: preprint molto recenti, abstract + PDF.")
            print("- OpenAlex: copertura più ampia + segnali come citazioni/anno.")
            print("Step successivo: usare un LLM (locale) nel client per fondere i risultati in una risposta unica.")
            return

        # --- LLM finale (Ollama) ---
        prompt = f"""
Sei un assistente di ricerca scientifica.

REGOLE OBBLIGATORIE:
1. Usa ESCLUSIVAMENTE le informazioni presenti nelle FONTI fornite.
2. OGNI affermazione fattuale nel testo DEVE avere una citazione inline
   nel formato [arXiv-1], [arXiv-2], [OpenAlex-1], ecc.
3. Se un'informazione NON è supportata dalle fonti, scrivi esplicitamente:
   "Non emergono evidenze dalle fonti fornite."
4. Non usare conoscenza generale o memoria esterna.

DOMANDA:
{args.question}

RISULTATI arXiv (lista, in ordine):
{json.dumps(ax_items, ensure_ascii=False, indent=2)}

RISULTATI OpenAlex (lista, in ordine):
{json.dumps(ox_items, ensure_ascii=False, indent=2)}

OUTPUT RICHIESTO:
- Un paragrafo di sintesi con CITAZIONI INLINE obbligatorie
- Una sezione finale "Fonti" che elenca:
  * [arXiv-i] titolo + link PDF
  * [OpenAlex-j] titolo + DOI o link OpenAlex
  
"""

        try:
            r = httpx.post(
                OLLAMA_URL,
                json={"model": args.model, "prompt": prompt, "stream": False},
                timeout=120.0
            )
            r.raise_for_status()
            data = r.json()
            answer = data.get("response", "").strip()
        except Exception as e:
            print("\n[ERRORE LLM] Non riesco a chiamare Ollama.")
            print("Assicurati che 'ollama serve' sia attivo e che il modello sia scaricato (es: ollama pull llama3.2).")
            print("Dettagli:", e)
            return

        print("\n==============================")
        print("RISPOSTA (LLM)")
        print("==============================")
        print(answer)

if __name__ == "__main__":
    asyncio.run(main())
'''

ROOT = Path(".").resolve()
(CLI := ROOT / "cli").mkdir(parents=True, exist_ok=True)
(CLI / "mcp_chat.py").write_text(client_cli, encoding="utf-8")
print("Creato:", CLI / "mcp_chat.py")


Creato: /Users/monica.costantini/Documents/workspace/mcp-demo/cli/mcp_chat.py


## 0.4 – Configurazione VS Code (MCP servers)

Crea `.vscode/mcp.json` così VS Code può avviare i server MCP locali (stdio).


In [77]:

from pathlib import Path
import json

ROOT = Path(".").resolve()
VSCODE = ROOT/".vscode"
VSCODE.mkdir(parents=True, exist_ok=True)

mcp_json = {
  "servers": {
    "arxiv-local": {"command": "python", "args": ["servers/arxiv_server.py"]},
    "openalex-local": {"command": "python", "args": ["servers/openalex_server.py"]}
  }
}

(VSCODE/"mcp.json").write_text(json.dumps(mcp_json, indent=2), encoding="utf-8")
print("Creato:", VSCODE/"mcp.json")


Creato: /Users/monica.costantini/Documents/workspace/mcp-demo/.vscode/mcp.json


## 0.5 – Dipendenze

Nel terminale di VS Code (consigliato), dentro `mcp-demo/`:

```bash
python -m venv .venv
# mac/linux
source .venv/bin/activate
# windows
# .venv\Scripts\activate

pip install -U pip
pip install "mcp[cli]" fastmcp arxiv httpx
```

Poi passa al Notebook 1 per testare il client CLI.
